# Metadata
```
Author: Alice Bogdan (ucn3qn@virginia.edu)
Course: DS 5001
Date: 3 May 2022
Semester Project
Topic: Gilmore Girls
```

Purpose: Using ETA on Gilmore Girls tv scripts. Analysis of preprocessed data

# Set Up

In [6]:
#libraries
import pandas as pd
import numpy as np
import nltk
# from sklearn.decomposition import PCA
from scipy.linalg import norm
import plotly.express as px
import seaborn as sns; sns.set()
import re
import scipy.cluster.hierarchy as sch
import matplotlib.pyplot as plt
from gensim.models import word2vec
from sklearn.manifold import TSNE
from sklearn.decomposition import LatentDirichletAllocation as LDA
from scipy.linalg import norm
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer #either just vectorizer or count/transformer in tandem

In [7]:
OHCO = ['Season','Episode','line_num','Character','sent_num','token_num']

In [8]:
#for convenience
SEAS = OHCO[:1]
EPI = OHCO[:2]
LINE = OHCO[:3]
CHAR = OHCO[:4]
SENT = OHCO[:5]
TOKEN = OHCO[:6]

In [9]:
#gradient colormap 
gradient_cmap = 'YlGnBu'

In [10]:
import os
import sys

In [11]:
#import libraries in local directory
sys.path.append('lib')
import hw07

In [52]:
ngram_range = (2,2) #countvectorize - pull out n-grams. Part of token list
n_terms = 10000 #subset vocab
pos_list = "NN NNS".split() #nouns only

# Load Data 

In [13]:
data_home = "data"
data_prefix = 'Gilmore_Girls'

In [14]:
#main data sets
DATA = pd.read_csv(f"{data_home}/output/{data_prefix}-DATA.csv")
#CORPUS = pd.read_csv(f"{data_home}/output/{data_prefix}-TOKENS.csv") #aka the CORPUS
#VOCAB = pd.read_csv(f"{data_home}/output/{data_prefix}-VOCAB.csv")
VOCAB2 = pd.read_csv(f"{data_home}/output/{data_prefix}-VOCAB.csv").set_index('term_str')
CORPUS2 = pd.read_csv(f"{data_home}/output/{data_prefix}-CORPUS.csv").set_index(TOKEN) #aka the CORPUS

In [15]:
#libraries
LIB_SEASON = pd.read_csv(f"{data_home}/output/{data_prefix}-LIB-SEASON.csv").set_index(['Season'])
LIB_EPISODE = pd.read_csv(f"{data_home}/output/{data_prefix}-LIB-EPISODE.csv")

In [16]:
LIB_SEASON

,Show,Num_Episodes,Year,Num_Lines
Season,,,,
1,Gilmore Girls,21,2000,16031
2,Gilmore Girls,22,2001,18264
3,Gilmore Girls,22,2002,17699
4,Gilmore Girls,22,2003,17806
5,Gilmore Girls,22,2004,16737
6,Gilmore Girls,22,2005,16145
7,Gilmore Girls,22,2006,15188


In [17]:
#character info
CHAR_INFO = pd.read_csv(f"{data_home}/output/Character-Info.csv")

In [18]:
#remove first column
DATA = DATA[["Season", "Episode", "Title","time_raw","line_num","Character","lines","Year"]]

In [19]:
VOCAB2.head()

,n,n_chars,p,s,i,h,max_pos,n_pos,cat_pos,stop,stem_porter,stem_snowball,stem_lancaster,max_pos_group
term_str,,,,,,,,,,,,,,
i,53349,1,0.045899,21.786969,4.445394,0.204039,PRP,2,"{'PRP', 'NN'}",1,i,i,i,PR
you,44496,3,0.038282,26.121741,4.707179,0.180202,PRP,2,"{'PRP', 'NN'}",1,you,you,you,PR
the,31072,3,0.026733,37.407087,5.225240,0.139686,DT,2,"{'NN', 'DT'}",1,the,the,the,DT
to,26174,2,0.022519,44.407160,5.472720,0.123240,TO,2,"{'NN', 'TO'}",1,to,to,to,TO
s,25469,1,0.021912,45.636381,5.512112,0.120783,NN,1,{'NN'},1,s,s,s,NN


In [20]:
CORPUS2.head()

pos_tuple pos  \
Season Episode line_num Character sent_num token_num                         
1      1       5        LORELAI   0        0          ('Please', 'VB')  VB   
                                           1            ('Luke', 'NN')  NN   
                                  1        0          ('Please', 'VB')  VB   
                                           1          ('please', 'NN')  NN   
                                           2          ('please', 'NN')  NN   

                                                     token_str term_str  \
Season Episode line_num Character sent_num token_num                      
1      1       5        LORELAI   0        0            Please   please   
                                           1              Luke     luke   
                                  1        0            Please   please   
                                           1            please   please   
                                           2            please   please   

                                                     pos_group  
Season Episode line_num Character sent_num token_num            
1      1       5        LORELAI   0        0                VB  
                                           1                NN  
                                  1        0                VB  
                                           1                NN  
                                           2                NN

In [21]:
CHAR_INFO

,Character,Gender,Status,S1_Age,S2_Age,S3_Age,S4_Age,S5_Age,S6_Age,S7_Age
0,DEAN,M,Main,Teen,Teen,Teen,YA,YA,YA,YA
1,EMILY,F,Main,OA,OA,OA,OA,OA,OA,OA
2,MICHEL,M,Main,Adult,Adult,Adult,Adult,Adult,Adult,Adult
3,PARIS,F,Main,Teen,Teen,Teen,YA,YA,YA,YA
4,JASON,M,Main,Adult,Adult,Adult,Adult,Adult,Adult,Adult
...,...,...,...,...,...,...,...,...,...,...
89,SHERRY,F,Family,Adult,Adult,Adult,Adult,Adult,Adult,Adult
90,SASHA,F,Family,Adult,Adult,Adult,Adult,Adult,Adult,Adult
91,PAUL ANKA,M,Stars_Hollow,Adult,Adult,Adult,Adult,Adult,Adult,Adult
92,CAESAR,M,Stars_Hollow,Adult,Adult,Adult,Adult,Adult,Adult,Adult


In [22]:
LIB_EPISODE = LIB_EPISODE.reset_index()

In [23]:
LIB_EPISODE['label'] = LIB_EPISODE.apply(lambda x: f"{x.Season}x{x.Episode}–{x.Title[:10].upper()}", 1)

In [24]:
LIB_EPISODE = LIB_EPISODE.drop(columns = ['index'])
LIB_EPISODE = LIB_EPISODE.set_index(OHCO[:2])

# Create DOC

In [25]:
def gather_docs(CORPUS, ohco_level, term_col='term_str'):
    OHCO = CORPUS.index.names
    CORPUS[term_col] = CORPUS[term_col].astype('str')
    DOC = CORPUS.groupby(OHCO[:ohco_level])[term_col].apply(lambda x:' '.join(x)).to_frame('doc_str')
    return DOC

In [26]:
DOC = gather_docs(CORPUS2, 4)

In [27]:
DOC['n_tokens'] = DOC.doc_str.apply(lambda x: len(x.split()))

In [28]:
DOC

doc_str  \
Season Episode line_num Character                                                      
1      1       5        LORELAI                     please luke please please please   
               6        LUKE                 how many cups have you had this morning   
               7        LORELAI                                                 none   
               8        LUKE                                                    plus   
               9        LORELAI                             five but yours is better   
...                                                                              ...   
7      22      807      RORY                          well it wasn t very flattering   
               808      LORELAI    well how will people know you re the world s g...   
               809      RORY                                            i don t know   
               810      LORELAI         i guess they ll just have to read your stuff   
               811      RORY                                              i guess so   

                                   n_tokens  
Season Episode line_num Character            
1      1       5        LORELAI           5  
               6        LUKE              8  
               7        LORELAI           1  
               8        LUKE              1  
               9        LORELAI           5  
...                                     ...  
7      22      807      RORY              6  
               808      LORELAI          12  
               809      RORY              4  
               810      LORELAI          10  
               811      RORY              3  

[117540 rows x 2 columns]

# Method 1: CountVectorizer + TfidfTransformer

## Create DTM

In [53]:
count_engine = CountVectorizer(
    stop_words='english',
    ngram_range= ngram_range,
    max_features=n_terms
)

In [54]:
X = count_engine.fit_transform(DOC.doc_str)

In [55]:
#X.toarray()

In [56]:
#count_engine.get_feature_names()

In [57]:
DTM = pd.DataFrame(X.toarray(), 
                   columns=count_engine.get_feature_names(), 
                   index=DOC.index)

In [58]:
DTM

00 00  00 morning  000 years  10 00  10 30  \
Season Episode line_num Character                                               
1      1       5        LORELAI        0           0          0      0      0   
               6        LUKE           0           0          0      0      0   
               7        LORELAI        0           0          0      0      0   
               8        LUKE           0           0          0      0      0   
               9        LORELAI        0           0          0      0      0   
...                                  ...         ...        ...    ...    ...   
7      22      807      RORY           0           0          0      0      0   
               808      LORELAI        0           0          0      0      0   
               809      RORY           0           0          0      0      0   
               810      LORELAI        0           0          0      0      0   
               811      RORY           0           0          0      0      0   

                                   10 minutes  10 years  100 sure  100 years  \
Season Episode line_num Character                                              
1      1       5        LORELAI             0         0         0          0   
               6        LUKE                0         0         0          0   
               7        LORELAI             0         0         0          0   
               8        LUKE                0         0         0          0   
               9        LORELAI             0         0         0          0   
...                                       ...       ...       ...        ...   
7      22      807      RORY                0         0         0          0   
               808      LORELAI             0         0         0          0   
               809      RORY                0         0         0          0   
               810      LORELAI             0         0         0          0   
               811      RORY                0         0         0          0   

                                   11 00  ...  young girl  young lady  \
Season Episode line_num Character         ...                           
1      1       5        LORELAI        0  ...           0           0   
               6        LUKE           0  ...           0           0   
               7        LORELAI        0  ...           0           0   
               8        LUKE           0  ...           0           0   
               9        LORELAI        0  ...           0           0   
...                                  ...  ...         ...         ...   
7      22      807      RORY           0  ...           0           0   
               808      LORELAI        0  ...           0           0   
               809      RORY           0  ...           0           0   
               810      LORELAI        0  ...           0           0   
               811      RORY           0  ...           0           0   

                                   young man  young men  young people  \
Season Episode line_num Character                                       
1      1       5        LORELAI            0          0             0   
               6        LUKE               0          0             0   
               7        LORELAI            0          0             0   
               8        LUKE               0          0             0   
               9        LORELAI            0          0             0   
...                                      ...        ...           ...   
7      22      807      RORY               0          0             0   
               808      LORELAI            0          0             0   
               809      RORY               0          0             0   
               810      LORELAI            0          0             0   
               811      RORY               0          0             0   

                                   young

## Get VOCAB

In [59]:
VOCAB = DTM.sum().to_frame('n')

In [60]:
VOCAB.sort_index()

,n
00 00,5
00 morning,16
000 years,4
10 00,16
10 30,4
...,...
young woman,5
zach brian,7
zach don,5
zach zach,5


In [61]:
VOCAB['n_chars'] = VOCAB.apply(lambda x: len(x.name), 1)
VOCAB['n_tokens'] = VOCAB.apply(lambda x: len(x.name.split()), 1)

In [62]:
VOCAB.value_counts('n_tokens')

n_tokens
2    10000
dtype: int64

In [63]:
VOCAB

,n,n_chars,n_tokens
00 00,5,5,2
00 morning,16,10,2
000 years,4,9,2
10 00,16,5,2
10 30,4,5,2
...,...,...,...
young woman,5,11,2
zach brian,7,10,2
zach don,5,8,2
zach zach,5,9,2


In [64]:
VOCAB[VOCAB.n_tokens == 2].sample(30)

,n,n_chars,n_tokens
god mom,4,7,2
god smells,6,10,2
hi hello,7,8,2
like introduce,6,14,2
thank hey,6,9,2
oh logan,8,8,2
exactly like,9,12,2
headmaster charleston,25,21,2
sure know,9,9,2
college experience,5,18,2


## Create TFIDF

In [65]:
# TfidfTransformer?

In [66]:
tfidf_engine = TfidfTransformer(norm='l2', use_idf=True)

In [67]:
#tfidf_engine_smooth = TfidfTransformer(norm='l2', use_idf=True, smooth_idf = True)

In [68]:
X1 = tfidf_engine.fit_transform(DTM)

In [69]:
TFIDF = pd.DataFrame(X1.toarray(), columns=DTM.columns, index=DTM.index)

In [70]:
TFIDF

00 00  00 morning  000 years  10 00  10 30  \
Season Episode line_num Character                                               
1      1       5        LORELAI      0.0         0.0        0.0    0.0    0.0   
               6        LUKE         0.0         0.0        0.0    0.0    0.0   
               7        LORELAI      0.0         0.0        0.0    0.0    0.0   
               8        LUKE         0.0         0.0        0.0    0.0    0.0   
               9        LORELAI      0.0         0.0        0.0    0.0    0.0   
...                                  ...         ...        ...    ...    ...   
7      22      807      RORY         0.0         0.0        0.0    0.0    0.0   
               808      LORELAI      0.0         0.0        0.0    0.0    0.0   
               809      RORY         0.0         0.0        0.0    0.0    0.0   
               810      LORELAI      0.0         0.0        0.0    0.0    0.0   
               811      RORY         0.0         0.0        0.0    0.0    0.0   

                                   10 minutes  10 years  100 sure  100 years  \
Season Episode line_num Character                                              
1      1       5        LORELAI           0.0       0.0       0.0        0.0   
               6        LUKE              0.0       0.0       0.0        0.0   
               7        LORELAI           0.0       0.0       0.0        0.0   
               8        LUKE              0.0       0.0       0.0        0.0   
               9        LORELAI           0.0       0.0       0.0        0.0   
...                                       ...       ...       ...        ...   
7      22      807      RORY              0.0       0.0       0.0        0.0   
               808      LORELAI           0.0       0.0       0.0        0.0   
               809      RORY              0.0       0.0       0.0        0.0   
               810      LORELAI           0.0       0.0       0.0        0.0   
               811      RORY              0.0       0.0       0.0        0.0   

                                   11 00  ...  young girl  young lady  \
Season Episode line_num Character         ...                           
1      1       5        LORELAI      0.0  ...         0.0         0.0   
               6        LUKE         0.0  ...         0.0         0.0   
               7        LORELAI      0.0  ...         0.0         0.0   
               8        LUKE         0.0  ...         0.0         0.0   
               9        LORELAI      0.0  ...         0.0         0.0   
...                                  ...  ...         ...         ...   
7      22      807      RORY         0.0  ...         0.0         0.0   
               808      LORELAI      0.0  ...         0.0         0.0   
               809      RORY         0.0  ...         0.0         0.0   
               810      LORELAI      0.0  ...         0.0         0.0   
               811      RORY         0.0  ...         0.0         0.0   

                                   young man  young men  young people  \
Season Episode line_num Character                                       
1      1       5        LORELAI          0.0        0.0           0.0   
               6        LUKE             0.0        0.0           0.0   
               7        LORELAI          0.0        0.0           0.0   
               8        LUKE             0.0        0.0           0.0   
               9        LORELAI          0.0        0.0           0.0   
...                                      ...        ...           ...   
7      22      807      RORY             0.0        0.0           0.0   
               808      LORELAI          0.0        0.0           0.0   
               809      RORY             0.0        0.0           0.0   
               810      LORELAI          0.0        0.0           0.0   
               811      RORY             0.0        0.0           0.0   

                                   young

## Add stats to VOCAB

In [71]:
VOCAB['tfidf_mean'] = TFIDF.mean()
VOCAB['df'] = DTM[DTM > 0].count()
VOCAB['dfidf'] = VOCAB.df * np.log2(len(TFIDF)/VOCAB.df)

In [72]:
VOCAB.sort_values('tfidf_mean', ascending=False).head(30)

,n,n_chars,n_tokens,tfidf_mean,df,dfidf
don know,1445,8,2,0.006412,1391,8903.631833
oh god,597,6,2,0.003534,583,4463.121639
ve got,747,6,2,0.003340,710,5233.493115
oh yeah,476,7,2,0.002747,476,3783.235869
don want,647,8,2,0.002685,609,4623.828761
don think,438,9,2,0.002228,435,3513.895499
uh huh,211,6,2,0.001435,200,1839.787218
stars hollow,295,12,2,0.001403,278,2425.230635
didn know,243,9,2,0.001197,233,2092.015229
ll just,253,7,2,0.001095,250,2219.251998


In [73]:
VOCAB.index.name = 'term_str'
VOCAB

,n,n_chars,n_tokens,tfidf_mean,df,dfidf
term_str,,,,,,
00 00,5,5,2,0.000027,5,72.604321
00 morning,16,10,2,0.000091,16,205.484676
000 years,4,9,2,0.000023,4,59.371169
10 00,16,5,2,0.000090,14,182.496123
10 30,4,5,2,0.000021,4,59.371169
...,...,...,...,...,...,...
young woman,5,11,2,0.000034,5,72.604321
zach brian,7,10,2,0.000047,6,85.546979
zach don,5,8,2,0.000030,5,72.604321


In [74]:
VOCAB.to_csv(f'{data_home}/output/{data_prefix}-VOCAB-CHAR-BIGRAM.csv')

# Create BOW

In [75]:
BOW = DTM[DTM > 0].stack().to_frame('n').join(TFIDF[TFIDF > 0].stack().to_frame('tfidf'))

In [76]:
BOW

n     tfidf
Season Episode line_num Character term_str                     
1      1       14       LORELAI   ve got          1.0  1.000000
               18       JOEY      look really     1.0  0.614003
                                  make look       1.0  0.630740
                                  really good     1.0  0.474518
               19       LORELAI   good best       1.0  0.701445
...                                               ...       ...
7      22      808      LORELAI   world greatest  1.0  0.765277
               809      RORY      don know        1.0  1.000000
               810      LORELAI   guess ll        1.0  0.569572
                                  just read       1.0  0.671096
                                  ll just         1.0  0.474572

[102716 rows x 2 columns]

In [77]:
BOW.to_csv(f'{data_home}/output/{data_prefix}-BOW-CHAR-BIGRAM.csv')

In [ ]:
#px.scatter(VOCAB.reset_index(), x='term_rank2', y='dfidf', color='max_pos', hover_name='term_str', height=500)

In [ ]:
#VOCAB3[VOCAB3['max_pos'] == "NN"].sample(20)

In [ ]:
BOW.sort_values('tfidf', ascending=False).head(20).style.background_gradient(cmap=gradient_cmap, high=.5)